## Crawling and Extracting Data from Websites

This module uses a set of non-standard libraries, which need to be installed on your machine. By default, your instance should have these installed, but if this is not the case, type these in the Unix shell prompt





In [ ]:
!sudo -H apt-get -y install libxml2-dev libxslt-dev python3-dev

and then

In [ ]:
!sudo -H pip3 install -U lxml

In [ ]:
!sudo -H pip3 install -U pandas

### Fetching the headlines from ESPN.com

Let's start by trying to fetch the headlines from the site ESPN.com.



In [1]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

# Let's start by fetching the page, and parsing it
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

In [2]:
doc

<Element html at 0x7f3ca41a8458>

Let's start by getting the content of the `<title>` node from the site:

In [3]:
# This will return back a list of nodes that match
# are called <title>......</title>
results = doc.xpath('//title/text()')
results

['ESPN: The Worldwide Leader in Sports']

In [4]:
results[0]

'ESPN: The Worldwide Leader in Sports'

In [5]:
results = doc.xpath('//title')
results

[<Element title at 0x7f3c7dcc1e08>]

In [7]:
# gives only the text stored directly under the node
results[0].text 

'ESPN: The Worldwide Leader in Sports'

In [8]:
# gives all the text stored directly under the node *and* its children
# for the <title> node, it does not make a difference
results[0].text_content() 

'ESPN: The Worldwide Leader in Sports'

In [11]:
# Compare the two results to see the difference
# The first one gives all the text stored under the root <HTML> node
# The second gives only the text immediately under HTML (but not under the children)
# doc.xpath("/html")[0].text_content()
# doc.xpath("/html")[0].text


'\n\t\n\t\t\n\n\n\nESPN: The Worldwide Leader in Sports\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t{\n\t\t"@context": "http://schema.org",\n\t\t"@type":    "WebSite",\n\t\t"name":     "ESPN",\n\t\t"url":      "http://www.espn.com/"\n\t}\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n;(function(){\nfunction rc(a){for(var b=a+"=",c=document.cookie.split(";"),d=0;d<c.length;d++){for(var e=c[d];" "===e.charAt(0);)e=e.substring(1,e.length);if(0===e.indexOf(b))return e.substring(b.length,e.length)}return null}var _nr=!1,_nrCookie=rc("_nr");null!==_nrCookie?"1"===_nrCookie&&(_nr=!0):Math.floor(100*Math.random())+1===13?(_nr=!0,document.cookie="_nr=1; path=/"):(_nr=!1,document.cookie="_nr=0; path=/"),_nr&&(window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var o=e[n]={exports:{}};t[n][0].call(o.exports,function(e){var o=t[n][1][e];return r(o||e)},o,o.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<n.length

The `doc` variable is an `HtmlElement` object, and we can now use **XPath** queries to locate the elements that we need. (Depending on time, we may do in class a tutorial on XPath. For now, you can look at the [W3Schools tutorial](http://www.w3schools.com/xpath/xpath_nodes.asp))

For example, to find all the `<a ...> ... </a>` tags in the returned html, which store the links in the page, we issue the command:

In [ ]:
<a href='http://www.stern.nyu.edu'>NYU Stern</a>

In [13]:
links = doc.xpath("//a")
len(links)

166

In [15]:
links = doc.xpath("//a")
# Iterates over all the links (this means all the nodes
# that matched the //a XPath query) and prints the content
# of the attribute href and the text for that node
for link in links:
    print("==================================")
    print(link.get("href"), "==>", link.text_content())

# ==> Menu
/ ==> ESPN
# ==> 
# ==> Scores
/nfl/ ==> NFL
/nba/ ==> NBA
/mlb/ ==> MLB
/college-football/ ==> NCAAF
/soccer/ ==> Soccer
/nhl/ ==> NHL
/golf/ ==> Golf
/tennis/ ==> Tennis
# ==> …
/mma/ ==> MMA
/wwe/ ==> WWE
/boxing/ ==> Boxing
/esports/ ==> esports
/chalk/ ==> Chalk
/analytics/ ==> Analytics
/mens-college-basketball/ ==> NCAAM
/womens-basketball/ ==> WNBA
/womens-basketball/ ==> NCAAW
http://www.espn.com/espnw/sport/ncaa-softball/ ==> NCAA Softball
/racing/nascar/ ==> NASCAR
/jayski/ ==> Jayski
/f1/ ==> F1
/racing/ ==> Racing
/olympics/ ==> Olympic Sports
/horse-racing/ ==> Horse
http://www.espn.com/college-sports/football/recruiting/ ==> RN FB
http://www.espn.com/college-sports/basketball/recruiting/ ==> RN BB
/college-sports/ ==> NCAA
/little-league-world-series/ ==> LLWS
http://www.espn.com/specialolympics/ ==> Special Olympics
http://xgames.com/ ==> X Games
http://espncricinfo.com/ ==> Cricket
/rugby/ ==> Rugby
http://www.tsn.ca/cfl ==> CFL
# ==> More ESPN
/fantasy/ ==>

In [16]:
lnk = links[70]
type(lnk)

lxml.html.HtmlElement

The `lnk` variable is again an HtmlElement. To get parts of the html element that we need, we can use the `get` method (e.g., to get the `href` attribute) and the `text` method (to get the text within the `<a>...</a>` tag.

In [17]:
lnk.get("href")

'/mlb/story/_/id/24942409/alex-bregman-houston-astros-annoyed-alds-schedule-glad-return-prime'

In [18]:
lnk.text_content()

"Annoyed Bregman: Astros should be 'main event'"

In [19]:
lnk.text_content().strip()

"Annoyed Bregman: Astros should be 'main event'"

Now, let's revisit the _list comprehension_ approach that we discussed in the Python Primer session, for quickly constructing lists:

In [22]:
urls = [lnk.get("href") for lnk in doc.xpath('//a')]
urls

['#',
 '/',
 '#',
 '#',
 '/nfl/',
 '/nba/',
 '/mlb/',
 '/college-football/',
 '/soccer/',
 '/nhl/',
 '/golf/',
 '/tennis/',
 '#',
 '/mma/',
 '/wwe/',
 '/boxing/',
 '/esports/',
 '/chalk/',
 '/analytics/',
 '/mens-college-basketball/',
 '/womens-basketball/',
 '/womens-basketball/',
 'http://www.espn.com/espnw/sport/ncaa-softball/',
 '/racing/nascar/',
 '/jayski/',
 '/f1/',
 '/racing/',
 '/olympics/',
 '/horse-racing/',
 'http://www.espn.com/college-sports/football/recruiting/',
 'http://www.espn.com/college-sports/basketball/recruiting/',
 '/college-sports/',
 '/little-league-world-series/',
 'http://www.espn.com/specialolympics/',
 'http://xgames.com/',
 'http://espncricinfo.com/',
 '/rugby/',
 'http://www.tsn.ca/cfl',
 '#',
 '/fantasy/',
 'http://www.espn.com/espnradio/index',
 'http://www.espn.com/watch/',
 'http://www.espn.com/watch/espnplus/?om-navmethod=topnav',
 'https:plus.espn.com/',
 'http://www.espn.com/watch/espnplus/',
 'http://www.espn.com/watch/collections/11264/nhl-live

#### Exercise

Use a list compresension approach, to get the text_content of all the URLs in the page.

In [24]:
# your code here


['Menu',
 'ESPN',
 '',
 'Scores',
 'NFL',
 'NBA',
 'MLB',
 'NCAAF',
 'Soccer',
 'NHL',
 'Golf',
 'Tennis',
 '…',
 'MMA',
 'WWE',
 'Boxing',
 'esports',
 'Chalk',
 'Analytics',
 'NCAAM',
 'WNBA',
 'NCAAW',
 'NCAA Softball',
 'NASCAR',
 'Jayski',
 'F1',
 'Racing',
 'Olympic Sports',
 'Horse',
 'RN FB',
 'RN BB',
 'NCAA',
 'LLWS',
 'Special Olympics',
 'X Games',
 'Cricket',
 'Rugby',
 'CFL',
 'More ESPN',
 'Fantasy',
 'Listen',
 'Watch',
 'ESPN+',
 'Subscribe Now',
 'ESPN+ Home',
 'Watch 180+ NHL Games',
 "Kobe Bryant's Detail",
 '30 for 30 Archive',
 'MLB Playoffs',
 'Fantasy Basketball: Sign Up',
 '',
 'Manage Favorites',
 'ESPN Deportes',
 'The Undefeated',
 'espnW',
 'ESPNFC',
 'X Games',
 'SEC Network',
 'ESPN',
 'ESPN Fantasy',
 'Facebook',
 'Twitter',
 'Instagram',
 'Snapchat',
 'YouTube',
 'ESPN Daily Newsletter',
 'ESPN Daily Calendar',
 "Step into Kyrie's world of ridiculous dribble movesChris Forsberg",
 "Steelers' Brown sued after alleged incident",
 "Saints' Thomas rips Norm

And now create a list where we put together text content and the URL for each link

In [ ]:
# your code here


#### Let's get the headlines...

Now, let's examine how we can get the data from the website. The key is to understand the structure of the HTML, where the data that we need is stored, and how to fetch the elements. Then, using the appropriate XPath queries, we will get what we want.

In [25]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas # To create a dataframe

Let's start by fetching the page, and parsing it

In [26]:
url = "http://www.espn.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

By using the `"Right-Click > Inspect"` option of Chrome,
we right click on the headlines and select `"Inspect"`.
This opens the source code.
There we see that all under a `<div class="headlineStack">` tag.

In [ ]:
headlineNode = doc.xpath('//div[@class="headlineStack"]')

The result of that operation is a list with 6 elements.

In [ ]:
type(headlineNode)

In [ ]:
len(headlineNode)

Each headline is under a  `<li><a href="...."></a>` tag.
So, we get all the `<li><a ...>` tags within the `<div class="headlineStack">`
(which is stored in the "`headlineNode`" variable)

In [ ]:
headlines = headlineNode[1].xpath('.//li/a')
len(headlines)

Now, we have the nodes with the conent in the headlines variable.
We extract the text and the URL.

In [ ]:
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
data

And let's create our dataframe, so that we can have a better view

In [ ]:
dataframe = pandas.DataFrame(data)
dataframe

#### Of course, there are always more than one way to skin a cat...

Alternatively, if we did not want to restrict ourselves to just the first headline box, we could write an alternative query, to get back all the headlines, that appear in an XPath `//div[@class="headlineStack"]//li/a`:

In [ ]:
headlines = doc.xpath('//div[@class="headlineStack"]//li/a')
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
df = pandas.DataFrame(data)
df

In [50]:
headlines = doc.xpath('//a[@data-mptype="headline"]')
data = [{"Title": a.text_content(), "URL": a.get("href")} for a in headlines]
df = pandas.DataFrame(data)
df

,Title,URL
0,Injury-riddled Jags sign free-agent RB Charles,/nfl/story/_/id/24942611/jacksonville-jaguars-...
1,Steelers' Brown sued after alleged incident,/nfl/story/_/id/24942401/antonio-brown-sued-al...
2,Saints' Thomas rips Norman in Twitter tirade,/nfl/story/_/id/24940559/michael-thomas-calls-...
3,Annoyed Bregman: Astros should be 'main event',/mlb/story/_/id/24942409/alex-bregman-houston-...
4,Jones: Cowboys haven't had No. 1 WR in years,/nfl/story/_/id/24941622/jerry-jones-says-cowb...
5,Chargers owner Alex Spanos dies at 95,/nfl/story/_/id/24941255/los-angeles-chargers-...
6,Kiper & McShay: RB prospects for 2019 draft,/college-football/insider/story/_/id/24919864/...
7,Early NFL betting look for Week 6: Another big...,/chalk/insider/story/_/id/24941619/another-big...
8,Exploring the complex nature of love as it rel...,http://www.espn.com/watch/series/c31393a0-346f...
9,Injury-riddled Jags sign free-agent RB Charles,/nfl/story/_/id/24942611/jacksonville-jaguars-...


### In Class Example: Crawl BuzzFeed

* We will try to get the top articles that appear on Buzzfeed
* We will grab the link for the article, the text of the title, the description, and the editor.
* The results will be stored in a dataframe (we will see in detail what a dataframe is, in a couple of modules)


In [51]:
#your code here
import requests
from lxml import html

resp = requests.get("http://www.buzzfeed.com")
doc = html.fromstring(resp.text)

#### Solution for Buzzfeed (as of October 9, 2018)

In [ ]:
import requests # This command allows us to fetch URLs
from lxml import html # This module will allow us to parse the returned HTML/XML
import pandas

# Let's start by fetching the page, and parsing it
url = "http://www.buzzfeed.com/"
response = requests.get(url) # get the html of that url
doc = html.fromstring(response.text) # parse it and create a document

articleNodes = doc.xpath("//div[@data-buzzblock='story-card']") 

def parseArticleNode(article):
    headline = article.xpath(".//h2")[0].text_content()
    headline_link = article.xpath(".//a")[0].get("href")
    description = article.xpath(".//p")[0].text_content()
    editor = article.xpath(".//a[contains(@class,'card__meta__link')]/span")[0].text_content().strip()

    result = {
        "headline": headline,
        "URL" : headline_link,
        "description" : description,
        "editor" : editor
    }
    return result

data = [parseArticleNode(article) for article in articleNodes]
df = pandas.DataFrame(data)
df